In [47]:
# Cell 1
import arcpy

In [48]:
# Cell 2
print(arcpy.env.workspace)

H:/GEOG/Data


In [49]:
# Cell 3
folder = "H:/GEOG/Data"



In [50]:
# Cell 4
arcpy.env.workspace=folder

In [51]:
# Cell 5
print(arcpy.env.workspace)

H:/GEOG/Data


In [52]:
# Cell 6
fcList=arcpy.ListFeatureClasses()

In [53]:
# Cell 7
print(fcList)

['IowaCounties.shp', 'IowaCountiesFoodProgram.shp', 'NewFARate19.shp', 'NewFARate22.shp']


In [54]:
# Cell 8
fc='H:/GEOG/Data/IowaCountiesFoodProgram.shp'
# fc=r"H:/GEOG/Data/IAcountyneighbors.dbf"
fcFields = arcpy.ListFields(fc)
for field in fcFields:
    print(field.name)
type(fcFields)

FID
Shape
PERIMETER
DOMCountyI
FIPS
Shape_Leng
Shape_Area
FIPS_INT
CountyName
StateAbbr
CountyDisp
ID
County_Nam
FIPS_1
Primary_Po
Service_Ar
Households
Recip22
Recip19
Allotments
Primary_Co
Primary__1
Primary__2
FArate22
FArate19
Pop22
Pop19
Pop_Incr


<class 'list'>

In [55]:
# Cell 9
# Use Search Cursor to Find Num of Counties w/Pop <5000:
fc='H:/GEOG/Data/IowaCountiesFoodProgram.shp'
lowpoplist=[] # initialize empty list for small counties
fields = ['CountyName', 'Pop22']
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        if row[1] <= 6000: # identify counties w/low population
            lowpoplist.append(row[0]) # collect small pop. county names here
            
print(f"Low Pop. Counties: {lowpoplist}")

Low Pop. Counties: ['Audubon', 'Adams', 'Taylor', 'Ringgold']


In [56]:
# Cell 10
# Generate Table with each County's Neighbors:
arcpy.management.MakeFeatureLayer(fc, # read in shapefile
                                  "IAcountyneighbors") # name your new data table

arcpy.analysis.PolygonNeighbors("IAcountyneighbors", # take name of new data table...
                                r"/IAcountyneighbors.dbf", # make an output dbf
                                "CountyName") # where neighbor status is provided via county name

<Result 'H:/GEOG/Data\\\\IAcountyneighbors.dbf'>

In [57]:
# Cell 11
# Set local variables
inFeatures = r"H:/GEOG/Data/IAcountyneighbors.dbf"
joinTable = "CountyPopulations.csv"
joinField = "nbr_County"

joined_table = arcpy.management.AddJoin(inFeatures, joinField, 
                                        joinTable, joinField)

outputTable = "join_table"
# Export the joined data to a new standalone table
arcpy.conversion.TableToTable(joined_table, arcpy.env.workspace, outputTable)

<Result 'H:/GEOG/Data\\join_table.dbf'>

In [58]:
# Cell 12
# Let's do a simple calculation of the 
# new Food Assistance Rate in
# Adams County when taking in account all
# neighbors' pops & recipient counts:

#        | Adams | Adair  | Cass |  Montgom. | Page | Ringgold | Taylor | Union
# new_recip = 292 +   522  +  1313  +  1202  +   1359  +  400    +  493  +  1352 
# new_pop =  3611 + 7494 + 13104 + 10205 + 15143 + 4670 + 5858 + 11887 

# new_FArate = new_recip / new_pop
# old_FArate = 0.080864
# print(new_FArate)
# print(old_FArate)
# print(new_pop)

In [59]:
# Cell 13
# Make a function that sums neighboring counties' populations OR recipient counts
# User specifies whether output is population ("pop") or recipient count ("recip")

def sum_nbr_counts(yourcounty, count_type):
# Step 1: Get List of Neighboring Counties
    fields1 = ["src_County", "nbr_County", "Pop22", "Recip22"]
    my_nbrs = set()
    with arcpy.da.SearchCursor("join_table.dbf", fields1) as cursor:
        for row in cursor:
            if row[0] == yourcounty:
                my_nbrs.add(row[1]) # append all neighboring counties
    nbrs_list = list(my_nbrs) # can coerce set into list
    nbrs_list.append(yourcounty) # we want to include yourcounty in new pop calculation
    # print(nbrs_list)
#--------------------------------------------------------------------
# Step 2: Sum Counts of Neighboring Counties in List from Step 1
    fields = ["CountyName", "Pop22", "Recip22"]
    outp = 0
    for i in range(len(nbrs_list)):
        with arcpy.da.SearchCursor('H:/GEOG/Data/IowaCountiesFoodProgram.shp', fields) as cursor:
            for row in cursor:
                if row[0] == nbrs_list[i]:
                    if count_type == "pop":
                        outp += row[1]
                        # print(row[1])
                    elif count_type=="recip":
                        outp += row[2]   
                        # print(row[2])
    return outp 

# Example with Adams County, get population count:
print(sum_nbr_counts("Adams", "pop"))

71972


In [60]:
# The below shows how to transfer your new FA rates to our original shapefile and plot

In [61]:
# Cell 14 (Optional)
# First, we make a couple new fields (columns) in prep for next step:
fields = ["src_County", "nbr_County", "Pop22", "Recip22"]
addfields = ["NewPop", "NewRecip", "NewFARate"]
newfields = fields + addfields

arcpy.management.AddField("join_table.dbf", "NewPop", "FLOAT")
arcpy.management.AddField("join_table.dbf", "NewRecip", "FLOAT")
arcpy.management.AddField("join_table.dbf", "NewFARate", "FLOAT")

<Result 'H:/GEOG/Data\\join_table.dbf'>

In [63]:
# Cell 15 (Optional)
# Make Cursor Iteratively update all low-population counties:

with arcpy.da.UpdateCursor("join_table.dbf", newfields) as cursor:
    for row in cursor:
        if row[0] in lowpoplist: # if source county (src_County) is low pop,
                row[4] = sum_nbr_counts(row[0], "pop") # calc new pop
                row[5] = sum_nbr_counts(row[0], "recip") # calc new recip count
                row[6] = row[5]/row[4]
                cursor.updateRow(row)

In [64]:
# Cell 16 (Optional)
# Keep only 1 row per source county (this will help us join with shp file)
arcpy.DeleteIdentical_management("join_table.dbf", ["src_County"])

<Result 'H:/GEOG/Data\\join_table.dbf'>

In [45]:
# Cell 17 (Optional)
fields = ["FArate22", "NewFARate"]
with arcpy.da.UpdateCursor("NewFARate22", fields) as cursor:
        for row in cursor:
            if row[1]!=0: # if NewFARate22 is nonzero,
                row[0]=row[1] # replace old Food Assist rate w/NewFARate
                cursor.updateRow(row)